# 문자열 생성 (Text Generation) RNN 모델

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

### 텐서플로와 다른 라이브러리 임포트

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import time

In [ ]:
# 런타임에서 할당하는데 필요한 양만큼의 GPU 메모리를 할당
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
    print(e)

### 셰익스피어 데이터셋 다운로드

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

### 데이터 전처리

#### 데이터 로딩

In [ ]:
# 읽은 다음 파이썬 2와 호환되도록 디코딩합니다.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# 텍스트의 길이는 그 안에 있는 문자의 수입니다.
print ('텍스트의 길이: {}자'.format(len(text)))

In [ ]:
# 텍스트의 처음 250자를 살펴봅니다
print(text[:250])

#### Vocabulary 생성

In [ ]:
# 파일의 고유 문자수를 출력합니다.
vocab = sorted(set(text))
print ('고유 문자수 {}개'.format(len(vocab)))

####  문자 별 인덱스, 인덱스 별 문자 맵핑 생성

In [ ]:
# 문자에서 인덱스로 매핑
char2idx = {u:i for i, u in enumerate(vocab)}

# 인덱스에서 문자로 매핑
idx2char = np.array(vocab)

#### 생성된 맵핑 확인 : char2idx 20개 항목 확인

In [ ]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

#### 문자열 데이터를 숫자열 데이터로 변환

In [ ]:
text_as_int = np.array([char2idx[c] for c in text])

#### 문자열에서 숫자열로 맵핑 확인

In [ ]:
# 텍스트에서 처음 13개의 문자가 숫자로 어떻게 매핑되었는지를 보여줍니다
print ('{} ---- 문자들이 다음의 정수로 매핑되었습니다 ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

### 데이터셋 생성

####  1. 문자 단위 데이터셋 생성

In [ ]:
# RNN 입력 sequence 길이
seq_length = 100

# 데이터셋 만들기
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# 처음 5개 문자 확인
for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

#### 2. 청크 단위 데이터셋 생성

In [ ]:
# label 생성을 위해 배치 길이를 seq_length+1 설정
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

# 처음 5개 sequence 확인
for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

#### 3. 입력과 타겟이 분리된 데이터셋 생성

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

#### 입력 & 타겟 확인

In [ ]:
for input_example, target_example in  dataset.take(1):
  print ('입력 데이터: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('타깃 데이터: ', repr(''.join(idx2char[target_example.numpy()])))

#### 4. 배치 단위의 데이터셋 생성

In [ ]:
BATCH_SIZE = 64 # 배치 크기
BUFFER_SIZE = 10000 # 데이터셋을 섞을 버퍼 크기

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
dataset

## 모델 정의

#### 모델 정의

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

#### 모델 생성

In [ ]:
vocab_size = len(vocab) # 어휘 사전의 크기
embedding_dim = 256     # 임베딩 차원
rnn_units = 1024        # RNN 유닛(unit) 개수

model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

#### 출력 Tensor Shape 확인

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (배치 크기, 시퀀스 길이, 어휘 사전 크기)")

#### 모델 구조 확인

In [ ]:
model.summary()

#### 예측 분포에서 샘플링 테스트
Categorical Distribution 샘플링

In [ ]:
# (bach size, number of class)형태의 2D Tensor로 입력
# 따라서, 100개의 timestep이 batch인 것으로 처리됨
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

# 출력이 (100,1)이므로 (100)으로 변경
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

100개 timestep에 대한 샘플

In [ ]:
sampled_indices

예측된 텍스트 복호화

In [ ]:
print("입력: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("예측된 다음 문자: \n", repr("".join(idx2char[sampled_indices])))

## 모델 훈련

#### 모델 컴파일

In [ ]:
model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

#### 체크포인트 콜백 정의

In [ ]:
# 체크포인트가 저장될 디렉토리
checkpoint_dir = './training_checkpoints'
# 체크포인트 파일 이름
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

#### 모델 훈련

In [ ]:
EPOCH = 30

# 체크포인트 콜백 설정
model.fit(dataset, epochs=EPOCH, callbacks=[checkpoint_callback])

## 모델 테스트
모델을 테스트 하기 위해 훈련된 가중치를 갖고 배치 크기가 1인 입력을 받는 모델을 새로 만들어야 함

#### 테스트 용 모델을 새로 생성 (단, batch_size=1로 설정)
Hint : build_model() 함수를 사용해서 생성할 것

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

#### 모델에 마지막 저장 체크포인트 복구 (Hint : model.load_weights() 이용)


In [ ]:
last_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
print(last_checkpoint)
model.load_weights(last_checkpoint)

#### 모델의 input shape을 [1, None]으로 변경

In [ ]:
# 배치 크기 1로 모델을 새로 빌드
model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

#### 모델 테스트

In [ ]:
# 학습된 모델을 사용하여 텍스트 생성
def generate_text(model, start_string, num_generate):
  
  num_generate = 1000 # 생성할 문자의 수

  # 시작 문자열을 숫자열로 변환
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0) # 2차원 배열로 변환

  text_generated = [] # 생성된 결과를 저장할 빈 문자열

  # temperature로 확률 값 조정 – 크면 균등분포, 낮으면 argmax와 같이 됨
  temperature = 1.0

  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval) # 배치 크기 = 1
      predictions = tf.squeeze(predictions, 0) # 배치 차원 제거
      predictions = predictions / temperature # temperature 적용
    
    
      # 범주형 분포를 사용하여 모델에서 리턴한 단어 예측
      # input   : [batch_size, num_classes]  RNN sequence를 batch 형태로 입력
      # output : [batch_size, num_samples] [-1,0]는 마지막 batch 항목에서 첫번째로 sampling한 값을 의미
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # 예측된 단어를 다음 입력으로 모델에 전달
      input_eval = tf.expand_dims([predicted_id], 0) # 2차원 배열로 변환

      text_generated.append(idx2char[predicted_id]) # 생성된 문자열에 추가

  return (start_string + ''.join(text_generated))

In [ ]:
num_generate = 1000 # 생성할 문자의 수

print(generate_text(model, start_string=u"ROMEO: ", num_generate=num_generate))